<a href="https://colab.research.google.com/github/RafsanJany-44/ARC_5SSC_Experiment/blob/main/SleepStage_EEG_EXperiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


###Function

In [2]:
!pip install imbalanced-learn
!pip install ipywidgets


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 69.0 MB/s eta 0:00:00


In [3]:
from imblearn.over_sampling import SMOTE
import ipywidgets as widgets
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

In [4]:
def dataset_balance(X_temp, y_temp):
  smote = SMOTE()
  X_temp, y_temp= smote.fit_resample(X_temp, y_temp)
  return pd.concat([pd.DataFrame(X_temp), pd.DataFrame(y_temp)], axis=1)

def Xy_balance(X_temp, y_temp):
  smote = SMOTE()
  return smote.fit_resample(X_temp, y_temp)

def Search_Null(dataset):
  dic={}
  for col in list(dataset.columns):
    rows = []
    flage = 0
    for row in range(dataset.shape[0]):
      if str(dataset[col][row]) == "nan":
        rows.append(row)
        flage = 1
    if flage ==1:
      dic[col] = rows
  
  return dic


def models_check_box(models):
  from IPython.display import display
  new_keys=[]
  for i in models:
    i=widgets.Checkbox(
      value=False,
      description=str(i),
      disabled=False,
      indent=False
      )
    display(i)
    new_keys.append(i)
  return new_keys


def StandardScaleData(data):
  scaler = StandardScaler()
  scaler.fit(data)
  return  scaler.transform(data)


def MinMaxScaleData(data):
  scaler = MinMaxScaler()
  scaler.fit(data)
  return scaler.transform(data)

#Starting

In [5]:
dataset = pd.read_csv("/content/drive/MyDrive/Sleep Stage 5 class/EEG_HMC_FeatureExtraction_2023.01.19.csv")
print(dataset.shape)
dataset.dropna(axis = 0,how='any', inplace = True)
dataset = dataset.reset_index(drop = True)
"""df.to_csv("New_EEG_NullValueRemoved_HMC.csv",index = False)
dataset = pd.read_csv("New_EEG_NullValueRemoved_HMC.csv")"""
target = "Sleep Stage"
result = {}
print(dataset.shape)

#Encoding
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
dataset[target]=encoder.fit_transform(dataset[target])

(108920, 52)
(108450, 52)


###Spliting into X and y

In [6]:
X =  dataset.loc[:,dataset.columns != target]  # removing Sleep Stage
X =  X.loc[:,X.columns != "Subject"]            # removing Status
X =  X.loc[:,X.columns != "Epoch"]             # removing Epoch
y = dataset[target]

#Feature Selection

In [ ]:
number_of_feat = 30

###ANOVA with f classifciation

In [ ]:
from sklearn.datasets import make_classification
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
import pandas as pd



fs = SelectKBest(score_func=f_classif, k=5)
fit = fs.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)

featureScores = pd.concat([dfcolumns,dfscores],axis=1)

featureScores.columns = ['Best_columns','Score_ANOVA'] 

lyst = featureScores.nlargest(number_of_feat,'Score_ANOVA')

#lyst.to_csv('Filter_Method_ANOVA_with_f_classif.csv')

list_of_feat = list(lyst["Best_columns"])

###Embedded Method

In [ ]:
''''
from sklearn.linear_model import LassoCV
reg = LassoCV()
reg.fit(X, y)
print("Best alpha using built-in LassoCV: %f" % reg.alpha_)
print("Best score using built-in LassoCV: %f" %reg.score(X,y))
coef = pd.Series(reg.coef_, index = X.columns)

print("Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")

imp_coef = coef.sort_values()

list_of_feat=[]


for i in range(coef.shape[0]):
  if coef[i]!=0:
    list_of_feat.append(dataset.iloc[:0,i+3].name)
    
df = pd.DataFrame(list_of_feat, columns=['Best_Features'])

#df.to_csv("Embedded_Method.csv")

list_of_feat = list(df["Best_Features"])
if number_of_feat < len(list_of_feat):
  list_of_feat = list_of_feat[:number_of_feat]
'''

###Pearson's with f regression

In [ ]:
'''
from sklearn.datasets import make_regression
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
import pandas as pd


fs = SelectKBest(score_func=f_regression, k=5)
fit = fs.fit(X,y)

dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(dataset.columns)
featureScores = pd.concat([dfcolumns,dfscores],axis=1)

featureScores.columns = ['Best_columns','Score_pearsons'] 


lyst = featureScores.nlargest(number_of_feat,'Score_pearsons')

#lyst.to_csv('Filter_Method_Pearson’s_with_f_regression.csv')

list_of_feat = list(lyst["Best_columns"])
'''

###Sequential Feature Selection

In [ ]:
''''
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
sfs = SequentialFeatureSelector(knn, n_features_to_select=number_of_feat)
sfs.fit(X, y)
list_of_feat=[]
list_of_feat=list(sfs.get_feature_names_out(X.columns))

df = pd.DataFrame(list_of_feat, columns=['Best_Features'])

#df.to_csv("Filter_Method_Sequential_feat_Selection_KNN.csv")

list_of_feat = list(df["Best_Features"])
if number_of_feat < len(list_of_feat):
  list_of_feat = list_of_feat[:number_of_feat]
  '''

###Feature list

In [ ]:
dfcolumns = pd.DataFrame(list_of_feat)
print(dfcolumns)

#Tunning for Scalling and Data Balancing

##Smote : NO
##Scaling : NO

In [7]:
def tune_1():
  X_new = X
  y_new = y
  return train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)

##Smote : NO
##Scaling : All Data (Standard)

In [8]:
def tune_2():
  X_new = StandardScaleData(X)
  y_new = y
  return train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)

##Smote : NO
##Scaling : All Data (MinMax)

In [9]:
def tune_3():
  X_new = MinMaxScaleData(X)
  y_new = y
  return train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)

##Smote : NO
##Scaling : Train (Stadard)

In [10]:
def tune_4():
  X_new = X
  y_new = y
  X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)
  X_train = StandardScaleData(X_train)
  return X_train, X_test, y_train, y_test

##Smote : NO
##Scaling : Train (MinMax)

In [11]:

def tune_5():
  X_new = X
  y_new = y
  X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)
  X_train = MinMaxScaleData(X_train)
  return X_train, X_test, y_train, y_test

##Smote : ALL
##Scaling : NO

In [12]:

def tune_6():
  X_new,y_new = Xy_balance(X,y)
  return train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)


##Smote : Train
##Scaling : NO

In [13]:

def tune_7():
  X_new = X
  y_new = y
  X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)
  X_train,y_train = Xy_balance(X_train,y_train)
  return X_train, X_test, y_train, y_test


##Smote : Train
##Scaling : Train (Standard)

In [14]:
def tune_8():
  X_new = X
  y_new = y
  X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)
  X_train,y_train = Xy_balance(X_train,y_train)
  X_train = StandardScaleData(X_train)
  return X_train, X_test, y_train, y_test

##Smote : Train
##Scaling : Train (MixMax)

In [15]:
def tune_9():
  X_new = X
  y_new = y
  X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)
  X_train,y_train = Xy_balance(X_train,y_train)
  X_train = MinMaxScaleData(X_train)
  return X_train, X_test, y_train, y_test

##Smote : All
##Scaling : Train (Standard)

In [16]:
def tune_10():
  X_new,y_new = Xy_balance(X,y)
  X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)
  X_train = StandardScaleData(X_train)
  return X_train, X_test, y_train, y_test 

##Smote : All
##Scaling : Train (MixMax)

In [17]:
def tune_11():
  X_new,y_new = Xy_balance(X,y)
  X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)
  X_train = MinMaxScaleData(X_train)
  return X_train, X_test, y_train, y_test 

##Smote : All
##Scaling : All (Standard)

In [18]:
def tune_12():
  X_new,y_new = Xy_balance(X,y)
  X_new = StandardScaleData(X_new)
  return train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)

##Smote : All
##Scaling : ALL (MinMax)

In [19]:
def tune_13():
  X_new,y_new = Xy_balance(X,y)
  X_new = MinMaxScaleData(X_new)
  return train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)

##Tunn Dictionary

In [20]:
tune_dic={
  "Smote : NO    ; Scaling : NO"                  : tune_1(),
  "Smote : NO    ; Scaling : All Data(Standard)"  : tune_2(),
  "Smote : NO    ; Scaling : All Data(MinMax)"    : tune_3(),
  "Smote : NO    ; Scaling : Train(Standard)"     : tune_4(),
  "Smote : NO    ; Scaling : Train(MinMax)"       : tune_5(),
  "Smote : ALL   ; Scaling : NO"                  : tune_6(),
  "Smote : Train ; Scaling : NO"                  : tune_7(),
  "Smote : Train ; Scaling : Train(Standard)"     : tune_8(),
  "Smote : Train ; Scaling : Train(MinMax)"       : tune_9(),
  "Smote : All   ; Scaling : Train(Standard)"     : tune_10(),
  "Smote : All   ; Scaling : Train(MinMax)"       : tune_11(),
  "Smote : All   ; Scaling : All(Standard)"       : tune_12(),
  "Smote : All   ; Scaling : All(Standard)"       : tune_13()
}



---



# **Training**

##ADABOOST

In [24]:
%%time
from sklearn.ensemble import AdaBoostClassifier
ada_defult = AdaBoostClassifier(random_state=0)

ada={}
for i in list(tune_dic.keys()):

  X_train, X_test, y_train, y_test = tune_dic[i]
  ada_defult.fit(X_train, y_train)
  y_pred = ada_defult.predict(X_test)

  from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
  accu = accuracy_score(y_test, y_pred)
  print(i)
  print("Accurecy: ",accu)
  ada[i] = accu
  print("------------------------------------------------------------------------------")
  print("------------------------------------------------------------------------------")

df = pd.DataFrame.from_dict(ada, orient="index")
df.to_csv("ada.csv")

Smote : NO    ; Scaling : NO
Accurecy:  0.5698939603503919
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(Standard)
Accurecy:  0.573997233748271
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(MinMax)
Accurecy:  0.5698939603503919
------------------------------------------------------------------------------
------------------------------------------------------------------------------


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but AdaBoostClassifier was fitted without feature names
  warnings.warn(


Smote : NO    ; Scaling : Train(Standard)
Accurecy:  0.17118487782388198
------------------------------------------------------------------------------
------------------------------------------------------------------------------


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but AdaBoostClassifier was fitted without feature names
  warnings.warn(


Smote : NO    ; Scaling : Train(MinMax)
Accurecy:  0.3953895804518211
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : ALL   ; Scaling : NO
Accurecy:  0.5421781475093842
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : Train ; Scaling : NO
Accurecy:  0.5260488704472107
------------------------------------------------------------------------------
------------------------------------------------------------------------------


...

Smote : Train ; Scaling : Train(MinMax)
Accurecy:  0.2995850622406639
------------------------------------------------------------------------------
------------------------------------------------------------------------------


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but AdaBoostClassifier was fitted without feature names
  warnings.warn(


Smote : All   ; Scaling : Train(Standard)
Accurecy:  0.19945216597341991
------------------------------------------------------------------------------
------------------------------------------------------------------------------


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but AdaBoostClassifier was fitted without feature names
  warnings.warn(


Smote : All   ; Scaling : Train(MinMax)
Accurecy:  0.3319722024956883
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : All   ; Scaling : All(Standard)
Accurecy:  0.5530080146089074
------------------------------------------------------------------------------
------------------------------------------------------------------------------
CPU times: user 7min 43s, sys: 466 ms, total: 7min 43s
Wall time: 7min 42s




---



---



##Graddient Boosting

In [25]:
%%time
from sklearn.ensemble import GradientBoostingClassifier
gradBoost_default = GradientBoostingClassifier(random_state=0)

grad={}

for i in list(tune_dic.keys()):

  X_train, X_test, y_train, y_test = tune_dic[i]
  gradBoost_default.fit(X_train, y_train)
  y_pred = gradBoost_default.predict(X_test)

  from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
  accu = accuracy_score(y_test, y_pred)
  print(i)
  print("Accurecy: ",accu)
  grad[i] = accu
  print("------------------------------------------------------------------------------")
  print("------------------------------------------------------------------------------")

df = pd.DataFrame.from_dict(grad, orient="index")
df.to_csv("grad.csv")

Smote : NO    ; Scaling : NO
Accurecy:  0.6614107883817427
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(Standard)
Accurecy:  0.6643153526970954
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(MinMax)
Accurecy:  0.6601659751037344
------------------------------------------------------------------------------
------------------------------------------------------------------------------


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but GradientBoostingClassifier was fitted without feature names
  warnings.warn(


Smote : NO    ; Scaling : Train(Standard)
Accurecy:  0.177731673582296
------------------------------------------------------------------------------
------------------------------------------------------------------------------


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but GradientBoostingClassifier was fitted without feature names
  warnings.warn(


Smote : NO    ; Scaling : Train(MinMax)
Accurecy:  0.4024896265560166
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : ALL   ; Scaling : NO
Accurecy:  0.6437303439180277
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : Train ; Scaling : NO
Accurecy:  0.6236514522821577
------------------------------------------------------------------------------
------------------------------------------------------------------------------


...

Smote : Train ; Scaling : Train(MinMax)
Accurecy:  0.3739972337482711
------------------------------------------------------------------------------
------------------------------------------------------------------------------


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but GradientBoostingClassifier was fitted without feature names
  warnings.warn(


Smote : All   ; Scaling : Train(Standard)
Accurecy:  0.19985796895607183
------------------------------------------------------------------------------
------------------------------------------------------------------------------


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but GradientBoostingClassifier was fitted without feature names
  warnings.warn(


Smote : All   ; Scaling : Train(MinMax)
Accurecy:  0.39418687227351124
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : All   ; Scaling : All(Standard)
Accurecy:  0.6445673125697474
------------------------------------------------------------------------------
------------------------------------------------------------------------------
CPU times: user 3h 15min 49s, sys: 8.5 s, total: 3h 15min 58s
Wall time: 3h 15min 37s


###Histogram-Based Gradient Boosting

In [26]:
%%time
from sklearn.ensemble import HistGradientBoostingClassifier

hisgradBoost_default = HistGradientBoostingClassifier(random_state=0)

hisgrad={}

for i in list(tune_dic.keys()):
  X_train, X_test, y_train, y_test = tune_dic[i]
  
  hisgradBoost_default.fit(X_train, y_train)
  y_pred = hisgradBoost_default.predict(X_test)

  from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
  accu = accuracy_score(y_test, y_pred)
  print(i)
  print("Accurecy: ",accu)
  hisgrad[i] = accu
  print("------------------------------------------------------------------------------")
  print("------------------------------------------------------------------------------")

df = pd.DataFrame.from_dict(hisgrad, orient="index")
df.to_csv("hisgrad.csv")

Smote : NO    ; Scaling : NO
Accurecy:  0.71572153065929
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(Standard)
Accurecy:  0.71572153065929
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(MinMax)
Accurecy:  0.71572153065929
------------------------------------------------------------------------------
------------------------------------------------------------------------------


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but HistGradientBoostingClassifier was fitted without feature names
  warnings.warn(


Smote : NO    ; Scaling : Train(Standard)
Accurecy:  0.2570308898109728
------------------------------------------------------------------------------
------------------------------------------------------------------------------


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but HistGradientBoostingClassifier was fitted without feature names
  warnings.warn(


Smote : NO    ; Scaling : Train(MinMax)
Accurecy:  0.3591055786076533
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : ALL   ; Scaling : NO
Accurecy:  0.7345541239728112
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : Train ; Scaling : NO
Accurecy:  0.6923005993545412
------------------------------------------------------------------------------
------------------------------------------------------------------------------


...

Smote : Train ; Scaling : Train(MinMax)
Accurecy:  0.27086214845550943
------------------------------------------------------------------------------
------------------------------------------------------------------------------


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but HistGradientBoostingClassifier was fitted without feature names
  warnings.warn(


Smote : All   ; Scaling : Train(Standard)
Accurecy:  0.17736126610530586
------------------------------------------------------------------------------
------------------------------------------------------------------------------


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but HistGradientBoostingClassifier was fitted without feature names
  warnings.warn(


Smote : All   ; Scaling : Train(MinMax)
Accurecy:  0.3922846707923303
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : All   ; Scaling : All(Standard)
Accurecy:  0.7357715329207669
------------------------------------------------------------------------------
------------------------------------------------------------------------------
CPU times: user 9min 21s, sys: 2.93 s, total: 9min 24s
Wall time: 4min 59s




---



---



##Random Forest

In [27]:
%%time
RanFor={}

from sklearn.ensemble import RandomForestClassifier
rf_default = RandomForestClassifier(random_state=0,n_jobs=-1)


for i in list(tune_dic.keys()):

  X_train, X_test, y_train, y_test = tune_dic[i]
  rf_default.fit(X_train, y_train)
  y_pred=rf_default.predict(X_test)


  from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
  accu = accuracy_score(y_test, y_pred)
  print(i)
  print("Accurecy: ",accu)
  RanFor[i] = accu
  print("------------------------------------------------------------------------------")
  print("------------------------------------------------------------------------------")

df = pd.DataFrame.from_dict(RanFor, orient="index")
df.to_csv("datRanFora.csv")

Smote : NO    ; Scaling : NO
Accurecy:  0.7256800368833564
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(Standard)
Accurecy:  0.7291378515444905
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(MinMax)
Accurecy:  0.7277086214845551
------------------------------------------------------------------------------
------------------------------------------------------------------------------


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


Smote : NO    ; Scaling : Train(Standard)
Accurecy:  0.16629783310281235
------------------------------------------------------------------------------
------------------------------------------------------------------------------


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


Smote : NO    ; Scaling : Train(MinMax)
Accurecy:  0.4782388197325957
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : ALL   ; Scaling : NO
Accurecy:  0.8228162726996043
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : Train ; Scaling : NO
Accurecy:  0.7103734439834025
------------------------------------------------------------------------------
------------------------------------------------------------------------------


...

Smote : Train ; Scaling : Train(MinMax)
Accurecy:  0.4773628400184417
------------------------------------------------------------------------------
------------------------------------------------------------------------------


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


Smote : All   ; Scaling : Train(Standard)
Accurecy:  0.19970579283757736
------------------------------------------------------------------------------
------------------------------------------------------------------------------


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


Smote : All   ; Scaling : Train(MinMax)
Accurecy:  0.4250025362686416
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : All   ; Scaling : All(Standard)
Accurecy:  0.8188596936187481
------------------------------------------------------------------------------
------------------------------------------------------------------------------
CPU times: user 30min 18s, sys: 1.08 s, total: 30min 19s
Wall time: 15min 39s




---



---



##XGB

In [28]:
%%time
import xgboost as xgb
xgb_deafult = xgb.XGBClassifier(random_state=0,n_jobs=-1)

xgb={}

for i in list(tune_dic.keys()):

  X_train, X_test, y_train, y_test = tune_dic[i]
  xgb_deafult.fit(X_train,y_train)
  y_pred = xgb_deafult.predict(X_test)


  from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
  accu = accuracy_score(y_test, y_pred)
  print(i)
  print("Accurecy: ",accu)
  xgb[i] = accu
  print("------------------------------------------------------------------------------")
  print("------------------------------------------------------------------------------")

df = pd.DataFrame.from_dict(xgb, orient="index")
df.to_csv("xgb.csv")

Smote : NO    ; Scaling : NO
Accurecy:  0.6432918395573998
------------------------------------------------------------------------------
------------------------------------------------------------------------------


AttributeError: ignored

##KNN

In [29]:
%%time
from sklearn.neighbors import KNeighborsClassifier
knn_default = KNeighborsClassifier(n_jobs = -1)

knn={}
for i in list(tune_dic.keys()):

  X_train, X_test, y_train, y_test = tune_dic[i]
  knn_default.fit(X_train, y_train)
  y_pred=knn_default.predict(X_test)


  from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
  accu = accuracy_score(y_test, y_pred)
  print(i)
  print("Accurecy: ",accu)
  knn[i] = accu
  print("------------------------------------------------------------------------------")
  print("------------------------------------------------------------------------------")

df = pd.DataFrame.from_dict(knn, orient="index")
df.to_csv("knn.csv")

Smote : NO    ; Scaling : NO
Accurecy:  0.5696173351775011
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(Standard)
Accurecy:  0.6390963577685569
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(MinMax)
Accurecy:  0.6217611802674043
------------------------------------------------------------------------------
------------------------------------------------------------------------------


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(


Smote : NO    ; Scaling : Train(Standard)
Accurecy:  0.26924850161364683
------------------------------------------------------------------------------
------------------------------------------------------------------------------


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(


Smote : NO    ; Scaling : Train(MinMax)
Accurecy:  0.2653296449976948
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : ALL   ; Scaling : NO
Accurecy:  0.7321700314497311
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : Train ; Scaling : NO
Accurecy:  0.5191793453204242
------------------------------------------------------------------------------
------------------------------------------------------------------------------


...

Smote : Train ; Scaling : Train(MinMax)
Accurecy:  0.17916090364223144
------------------------------------------------------------------------------
------------------------------------------------------------------------------


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(


Smote : All   ; Scaling : Train(Standard)
Accurecy:  0.19960434209191438
------------------------------------------------------------------------------
------------------------------------------------------------------------------


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(


Smote : All   ; Scaling : Train(MinMax)
Accurecy:  0.20868418382875115
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : All   ; Scaling : All(Standard)
Accurecy:  0.7336157045754287
------------------------------------------------------------------------------
------------------------------------------------------------------------------
CPU times: user 46min 5s, sys: 54.3 s, total: 46min 59s
Wall time: 39min 59s


##NB

In [30]:
%%time
from sklearn.naive_bayes import GaussianNB

nb_deafult = GaussianNB()

nb={}
for i in list(tune_dic.keys()):

  X_train, X_test, y_train, y_test = tune_dic[i]
  nb_deafult.fit(X_train, y_train)
  y_pred = nb_deafult.predict(X_test)


  from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
  accu = accuracy_score(y_test, y_pred)
  print(i)
  print("Accurecy: ",accu)
  nb[i] = accu
  print("------------------------------------------------------------------------------")
  print("------------------------------------------------------------------------------")

df = pd.DataFrame.from_dict(nb, orient="index")
df.to_csv("nb.csv")

Smote : NO    ; Scaling : NO
Accurecy:  0.3595666205624712
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(Standard)
Accurecy:  0.31867219917012446
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(MinMax)
Accurecy:  0.31867219917012446
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : Train(Standard)
Accurecy:  0.10018441678192716
------------------------------------------------------------------------------
------------------------------------------------------------------------------


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but GaussianNB was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but GaussianNB was fitted without feature names
  warnings.warn(


Smote : NO    ; Scaling : Train(MinMax)
Accurecy:  0.11350852927616413
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : ALL   ; Scaling : NO
Accurecy:  0.20170944506442123
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : Train ; Scaling : NO
Accurecy:  0.1761180267404334
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : Train ; Scaling : Train(Standard)
Accurecy:  0.09944674965421853
------------------------------------------------------------------------------
------------------------------------------------------------------------------


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but GaussianNB was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but GaussianNB was fitted without feature names
  warnings.warn(


Smote : Train ; Scaling : Train(MinMax)
Accurecy:  0.11318579990779161
------------------------------------------------------------------------------
------------------------------------------------------------------------------


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but GaussianNB was fitted without feature names
  warnings.warn(


Smote : All   ; Scaling : Train(Standard)
Accurecy:  0.18124175712691487
------------------------------------------------------------------------------
------------------------------------------------------------------------------


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but GaussianNB was fitted without feature names
  warnings.warn(


Smote : All   ; Scaling : Train(MinMax)
Accurecy:  0.20789794054986305
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : All   ; Scaling : All(Standard)
Accurecy:  0.35933854113827735
------------------------------------------------------------------------------
------------------------------------------------------------------------------
CPU times: user 2.55 s, sys: 21.8 ms, total: 2.58 s
Wall time: 2.53 s


##Support Vector Machines

####SVM

In [31]:
"""%%time
from sklearn import svm

svm = svm.SVC(decision_function_shape='ovo')

svm_svc={}
for i in list(tune_dic.keys()):

  X_train, X_test, y_train, y_test = tune_dic[i]
  svm.fit(X_train, y_train)
  y_pred = svm.predict(X_test)


  from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
  accu = accuracy_score(y_test, y_pred)
  print(i)
  print("Accurecy: ",accu)
  svm_svc[i] = accu
  print("------------------------------------------------------------------------------")
  print("------------------------------------------------------------------------------")

df = pd.DataFrame.from_dict(svm_svc, orient="index")
df.to_csv("svm_svc.csv")"""

'%%time\nfrom sklearn import svm\n\nsvm = svm.SVC(decision_function_shape=\'ovo\')\n\nsvm_svc={}\nfor i in list(tune_dic.keys()):\n\n  X_train, X_test, y_train, y_test = tune_dic[i]\n  svm.fit(X_train, y_train)\n  y_pred = svm.predict(X_test)\n\n\n  from sklearn.metrics import classification_report, confusion_matrix, accuracy_score\n  accu = accuracy_score(y_test, y_pred)\n  print(i)\n  print("Accurecy: ",accu)\n  svm_svc[i] = accu\n  print("------------------------------------------------------------------------------")\n  print("------------------------------------------------------------------------------")\n\ndf = pd.DataFrame.from_dict(svm_svc, orient="index")\ndf.to_csv("svm_svc.csv")'

####Linear

In [32]:
"""%%time
from sklearn import svm

LinearSVC = svm.LinearSVC()

lin_svc = {}
for i in list(tune_dic.keys()):

  X_train, X_test, y_train, y_test = tune_dic[i]
  LinearSVC.fit(X_train, y_train)
  y_pred = LinearSVC.predict(X_test)



  from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
  accu = accuracy_score(y_test, y_pred)
  print(i)
  print("Accurecy: ",accu)
  lin_svc[i] = accu
  print("------------------------------------------------------------------------------")
  print("------------------------------------------------------------------------------")

df = pd.DataFrame.from_dict(lin_svc, orient="index")
df.to_csv("lin_svc.csv")"""

'%%time\nfrom sklearn import svm\n\nLinearSVC = svm.LinearSVC()\n\nlin_svc = {}\nfor i in list(tune_dic.keys()):\n\n  X_train, X_test, y_train, y_test = tune_dic[i]\n  LinearSVC.fit(X_train, y_train)\n  y_pred = LinearSVC.predict(X_test)\n\n\n\n  from sklearn.metrics import classification_report, confusion_matrix, accuracy_score\n  accu = accuracy_score(y_test, y_pred)\n  print(i)\n  print("Accurecy: ",accu)\n  lin_svc[i] = accu\n  print("------------------------------------------------------------------------------")\n  print("------------------------------------------------------------------------------")\n\ndf = pd.DataFrame.from_dict(lin_svc, orient="index")\ndf.to_csv("lin_svc.csv")'

##Stochastic Gradient Descent

In [33]:
"""%%time
from sklearn.linear_model import SGDClassifier

SGDClassifier = SGDClassifier()

sgdc={}
for i in list(tune_dic.keys()):

  X_train, X_test, y_train, y_test = tune_dic[i]
  SGDClassifier.fit(X_train, y_train)
  y_pred = SGDClassifier.predict(X_test)


  from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
  accu = accuracy_score(y_test, y_pred)
  print(i)
  print("Accurecy: ",accu)
  sgdc[i] = accu
  print("------------------------------------------------------------------------------")
  print("------------------------------------------------------------------------------")

df = pd.DataFrame.from_dict(sgdc, orient="index")
df.to_csv("sgdc.csv")"""

'%%time\nfrom sklearn.linear_model import SGDClassifier\n\nSGDClassifier = SGDClassifier()\n\nsgdc={}\nfor i in list(tune_dic.keys()):\n\n  X_train, X_test, y_train, y_test = tune_dic[i]\n  SGDClassifier.fit(X_train, y_train)\n  y_pred = SGDClassifier.predict(X_test)\n\n\n  from sklearn.metrics import classification_report, confusion_matrix, accuracy_score\n  accu = accuracy_score(y_test, y_pred)\n  print(i)\n  print("Accurecy: ",accu)\n  sgdc[i] = accu\n  print("------------------------------------------------------------------------------")\n  print("------------------------------------------------------------------------------")\n\ndf = pd.DataFrame.from_dict(sgdc, orient="index")\ndf.to_csv("sgdc.csv")'

##Tree algorithms

###DecisionTreeRegressor

In [21]:
%%time
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier()

dt={}
for i in list(tune_dic.keys()):

  X_train, X_test, y_train, y_test = tune_dic[i]
  dtc.fit(X_train, y_train)
  y_pred = dtc.predict(X_test)


  from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
  accu = accuracy_score(y_test, y_pred)
  print(i)
  print("Accurecy: ",accu)
  dt[i] = accu
  print("------------------------------------------------------------------------------")
  print("------------------------------------------------------------------------------")

df = pd.DataFrame.from_dict(dt, orient="index")
df.to_csv("dt.csv")

Smote : NO    ; Scaling : NO
Accurecy:  0.598893499308437
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(Standard)
Accurecy:  0.6010603964960811
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(MinMax)
Accurecy:  0.6010603964960811
------------------------------------------------------------------------------
------------------------------------------------------------------------------


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(


Smote : NO    ; Scaling : Train(Standard)
Accurecy:  0.1297833102812356
------------------------------------------------------------------------------
------------------------------------------------------------------------------


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(


Smote : NO    ; Scaling : Train(MinMax)
Accurecy:  0.31945597049331487
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : ALL   ; Scaling : NO
Accurecy:  0.692173074972101
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : Train ; Scaling : NO
Accurecy:  0.5861687413554634
------------------------------------------------------------------------------
------------------------------------------------------------------------------


...

Smote : Train ; Scaling : Train(MinMax)
Accurecy:  0.2702166897187644
------------------------------------------------------------------------------
------------------------------------------------------------------------------


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(


Smote : All   ; Scaling : Train(Standard)
Accurecy:  0.2506594298468094
------------------------------------------------------------------------------
------------------------------------------------------------------------------


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(


Smote : All   ; Scaling : Train(MinMax)
Accurecy:  0.2835294714416151
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : All   ; Scaling : All(Standard)
Accurecy:  0.6922998884041798
------------------------------------------------------------------------------
------------------------------------------------------------------------------
CPU times: user 1min 54s, sys: 77.6 ms, total: 1min 54s
Wall time: 1min 54s


In [22]:
!pip install catboost

from catboost import CatBoostClassifier

#cat = CatBoostClassifier(task_type="GPU")

cat=CatBoostClassifier()

catb={}
for i in list(tune_dic.keys()):

  X_train, X_test, y_train, y_test = tune_dic[i]
  cat.fit(X_train,y_train)
  y_pred = cat.predict(X_test)

  from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
  accu = accuracy_score(y_test, y_pred)
  print(i)
  print("Accurecy: ",accu)
  catb[i] = accu
  print("------------------------------------------------------------------------------")
  print("------------------------------------------------------------------------------")

df = pd.DataFrame.from_dict(catb, orient="index")
df.to_csv("catb.csv")

Streaming output truncated to the last 5000 lines.
24:	learn: 1.0496452	total: 4.18s	remaining: 2m 42s
25:	learn: 1.0443890	total: 4.32s	remaining: 2m 41s
26:	learn: 1.0393940	total: 4.46s	remaining: 2m 40s
27:	learn: 1.0354785	total: 4.61s	remaining: 2m 39s
28:	learn: 1.0305932	total: 4.76s	remaining: 2m 39s
29:	learn: 1.0268235	total: 4.9s	remaining: 2m 38s
30:	learn: 1.0220432	total: 5.04s	remaining: 2m 37s
31:	learn: 1.0181502	total: 5.21s	remaining: 2m 37s
32:	learn: 1.0144505	total: 5.35s	remaining: 2m 36s
33:	learn: 1.0106470	total: 5.52s	remaining: 2m 36s
34:	learn: 1.0077048	total: 5.65s	remaining: 2m 35s
35:	learn: 1.0036249	total: 5.84s	remaining: 2m 36s
36:	learn: 1.0003922	total: 5.97s	remaining: 2m 35s
37:	learn: 0.9967124	total: 6.13s	remaining: 2m 35s
38:	learn: 0.9937691	total: 6.28s	remaining: 2m 34s
39:	learn: 0.9906617	total: 6.43s	remaining: 2m 34s
40:	learn: 0.9877207	total: 6.57s	remaining: 2m 33s
41:	learn: 0.9848629	total: 6.7s	remaining: 2m 32s
42:	learn: 0.98



---



---

